In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.style.use('ggplot')

In [2]:
car=pd.read_csv('quikr_car.csv')

In [3]:
car

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel
...,...,...,...,...,...,...
887,Ta,Tara,zest,"3,10,000",NaN,NaN
888,Tata Zest XM Diesel,Tata,2018,"2,60,000","27,000 kms",Diesel
889,Mahindra Quanto C8,Mahindra,2013,"3,90,000","40,000 kms",Diesel
890,Honda Amaze 1.2 E i VTEC,Honda,2014,"1,80,000",Petrol,NaN


In [5]:
car.isnull().sum()

name           0
company        0
year           0
Price          0
kms_driven    52
fuel_type     55
dtype: int64

In [6]:
car=car[~car['fuel_type'].isna()]
car=car[~car['kms_driven'].isna()]

In [7]:
car.isnull().sum()

name          0
company       0
year          0
Price         0
kms_driven    0
fuel_type     0
dtype: int64

In [8]:
car['kms_driven']=car['kms_driven'].str.split().str.get(0).str.replace(',','')

In [9]:
car=car[car['Price'] != 'Ask For Price']

In [10]:
car['Price']=car['Price'].str.replace(',','')

In [11]:
X=car.drop(columns='Price')
y=car['Price']

In [12]:
#split test and train data using train_test__split()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
X_test

,name,company,year,kms_driven,fuel_type
204,Maruti Suzuki Swift VXI BSIII,Maruti,2006,45000,Petrol
312,Honda City,Honda,2011,55000,Petrol
791,Chevrolet Beat LS Diesel,Chevrolet,2014,31000,Diesel
236,Datsun GO T O,Datsun,2016,7000,Petrol
570,Hyundai i10,Hyundai,2009,27000,Petrol
...,...,...,...,...,...
466,Hyundai Eon Era Plus,Hyundai,2018,2110,Petrol
827,Hyundai i20 Asta,Hyundai,2014,31000,Petrol
759,Hyundai Verna 1.6 CRDI SX Plus AT,Hyundai,2012,29000,Diesel
707,Maruti Suzuki Omni 8 STR BS III,Maruti,2009,56000,Petrol


In [14]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(drop='first',dtype=np.int32)
X_train_new=ohe.fit_transform(X_train[['name','company','fuel_type']]).toarray()
X_test_new=ohe.fit_transform(X_test[['name','company','fuel_type']]).toarray()

In [15]:
X_train_new

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [16]:
#X_train_new= np.hstack((X_train[['year','kms_driven']].values,X_train_new))
#X_test_new= np.hstack((X_test[['year','kms_driven']].values,X_test_new))

In [17]:
X_train_new

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [18]:
from sklearn.compose import make_column_transformer
column_trans=make_column_transformer((OneHotEncoder(handle_unknown='ignore'),['name','company','fuel_type']),
                                    remainder='passthrough')


In [19]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()

In [20]:
#lr.fit(X_train_new,y_train)


In [21]:
from sklearn.pipeline import make_pipeline
pipe=make_pipeline(column_trans,lr)

In [22]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [23]:

y_pred=pipe.predict(X_test)

In [24]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.18990922466862614

In [25]:
scores=[]
for i in range(1000):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [26]:
np.argmax(scores)

144

In [27]:
 scores[np.argmax(scores)]

0.8552250378917783

In [28]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=np.argmax(scores))
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
r2_score(y_test,y_pred)

0.8552250378917783

In [30]:
 pip install pickle-mixin

Note: you may need to restart the kernel to use updated packages.


In [31]:
pickle.dump(car,open('df.pkl','wb'))

In [29]:
import pickle
pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))

In [34]:
pipe.predict(pd.DataFrame(columns=['name','company','year','kms_driven','fuel_type'],data=np.array(['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']).reshape(1,5)))

array([454862.1441974])